# Bone Segmentation with Datamint and Pytorch Lightning

In this notebook, we will use [Datamint](https://sonanceai.github.io/datamint-python-api/) + [Pytorch Lightning](https://lightning.ai/docs/pytorch/stable/) to run a segmentation task on a 2D dataset of bone images.

## Table of Contents

1. [Introduction](#bone-segmentation-with-datamint-and-pytorch-lightning)
2. [Creating Experiments and Loading Datasets](#creating-experiments-and-loading-datasets)
3. [Quick Visualization](#quick-visualization)
4. [Easy Generation of Dataloaders](#easy-generation-of-dataloaders)
5. [Defining Our Model](#defining-our-model)
6. [Training](#training)
7. [Predict](#predict)


## Creating experiments and loading datasets

Datamint organizes ML workflows into experiments within projects. Below we create an experiment in the "BoneSeg" project:

In [1]:
from datamintapi import Experiment

exp = Experiment(
    name="Experiment 1",
    project_name="BoneSeg",
    allow_existing=True,  # allows connecting to an existing experiment with the same name
    auto_log=False,  # disables automatic logging.
    # dry_run=True, # uncomment to test without saving to the server
)

The `get_dataset()` method provides a standardized way to access data with immediate preprocessing options:

| Parameter | Purpose |
|-----------|----------|
| `return_as_semantic_segmentation` | Converts instance segmentation to semantic segmentation |
| `semantic_seg_merge_strategy` | How to combine multiple masks from different annotators |
| `return_frame_by_frame` | Process data frame-by-frame or video-by-video | 

> **TIP:**
> Check more handy transformations at [DatamintDataset documentation](https://sonanceai.github.io/datamint-python-api/datamint.dataset.html#datamintapi.dataset.dataset.DatamintDataset)

In [2]:
from torchvision.transforms import v2
from torchvision.models.segmentation import DeepLabV3_ResNet50_Weights

IMAGE_SIZE = (520, 520)

# Define image transformations - standardization is important for pretrained models
image_transform = v2.Compose([v2.Resize(IMAGE_SIZE),
                              v2.Grayscale(num_output_channels=3),
                              v2.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.0, hue=0.0),
                              DeepLabV3_ResNet50_Weights.DEFAULT.transforms(),
                              ])

# It is important to use NEAREST interpolation for masks.
mask_transform = v2.Resize(IMAGE_SIZE, antialias=False, interpolation=v2.InterpolationMode.NEAREST)

# Handy dataset parameters
dataset_params = dict(
    return_as_semantic_segmentation=True,  # Transforms instance segmentation into semantic segmentation.
    semantic_seg_merge_strategy="union",  # Merges all author's masks into a single one.
    return_frame_by_frame=True,  # Iterates over each frame of the video, instead of the video as a whole.
    image_transform=image_transform,  # Transforms the image.
    mask_transform=mask_transform,  # Transforms the mask.
    discard_without_annotations=True,  # Discards images without annotations.
)

# Get the train dataset associated with the experiment.
Dtrain = exp.get_dataset(
    **dataset_params,
    split='train'
)

# Get the test dataset associated with the experiment.
Dtest = exp.get_dataset(
    **dataset_params,
    split='test'
)
print(len(Dtrain), len(Dtest))

3 1


## Quick visualization

Let's visualize a sample from our dataset to confirm image and mask alignment. The `draw_masks()` function overlays segmentation masks on top of the original image:
 

In [ ]:
from datamintapi.utils.visualization import draw_masks, show

# Fetch a sample item from the training dataset
item = Dtrain[3]

segs = item['segmentations']  # Tensor of shape (L, H, W), where L is the number of labels plus background.
# segs[0] is the background mask

image = item['image']  # Tensor of shape (C, H, W)
segs_names = Dtrain.segmentation_labels_set  # (list of str)

# Display the image with bone segmentation masks overlaid
image_with_mask = draw_masks(image,
                             masks=segs[1:] == 1)
show(image_with_mask)

## Easy generation of dataloaders

DatamintDataset provides a convenient `get_dataloader()` method that wraps PyTorch's DataLoader with appropriate settings, so you don't need to provide a `collate_fn`, for instance.

In [ ]:
train_dataloader = Dtrain.get_dataloader(batch_size=8, num_workers=4, shuffle=True)
test_dataloader = Dtest.get_dataloader(batch_size=2, num_workers=4, shuffle=False)
# Check the dataloader output shapes
sample_batch = next(iter(train_dataloader))
print(sample_batch['image'].shape)  # (8, 3, 520, 520)
print(sample_batch['segmentations'].shape) # (8, 5, 520, 520)

## Defining our model

We'll create a PyTorch Lightning model with:

- **Architecture**: DeepLabV3 with ResNet50 backbone for semantic segmentation
- **Metrics**: A combination of segmentation metrics (IoU) and classification metrics adapted for segmentation tasks
- **Logging**: Integration with both Lightning and Datamint's experiment tracking

> **Key concept**: The `SegmentationToClassificationWrapper` allows us to use classification metrics like F1 and Precision with segmentation data

In [5]:
import lightning as L
import torchvision
from torch import nn
from torchvision.models.segmentation import deeplabv3_resnet50
from torchmetrics.segmentation import MeanIoU
from torchmetrics.classification import Recall, Precision, F1Score, Specificity, Accuracy
from datamintapi.utils.torchmetrics import SegmentationToClassificationWrapper
from torch import Tensor
import torch


class MyModel(L.LightningModule):
    """DeepLabV3 semantic segmentation model with ResNet50 backbone."""
    
    def __init__(self,
                 num_classes: int,
                 experiment: Experiment = None):
        """
        Initialize model with metrics for training, validation and testing.
        
        Args:
            num_classes: Number of segmentation classes (excluding background)
            experiment: Optional experiment object for logging to Datamint
        """
        super().__init__()
        # self.model = MyModel.initialize_model_mobilenet(num_classes+1,
        #                                                 weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT)
        self.experiment = experiment
        self.model = MyModel.initialize_model(num_classes+1)
        self.criterion = nn.CrossEntropyLoss()
        self.num_classes = num_classes

        # train metrics #
        self.train_cls_metrics = self.create_cls_metrics()
        self.train_seg_metrics = self.create_seg_metrics()

        # val metrics #
        self.val_cls_metrics = self.create_cls_metrics()
        self.val_seg_metrics = self.create_seg_metrics()

        # test metrics #
        self.test_cls_metrics = self.create_cls_metrics()
        self.test_seg_metrics = self.create_seg_metrics()

    def create_seg_metrics(self) -> nn.ModuleDict:
        """
        Create segmentation metrics (IoU).
        
        Returns:
            ModuleDict containing segmentation metrics
        """
        metrics = nn.ModuleDict()
        metrics["iou"] = MeanIoU(num_classes=self.num_classes+1, include_background=True)
        return metrics

    def create_cls_metrics(self) -> nn.ModuleDict:
        """
        Create classification metrics wrapped for segmentation evaluation.
        
        Returns:
            ModuleDict with classification metrics adapted for segmentation
        """
        metrics = nn.ModuleDict()
        for clsmetric_cls in [Recall, Precision, F1Score, Specificity, Accuracy]:
            clsmetric_obj = clsmetric_cls(task="multilabel", num_labels=self.num_classes+1, average="macro")
            metric = SegmentationToClassificationWrapper(clsmetric_obj, iou_threshold=0.5)
            metrics[clsmetric_cls.__name__] = metric

        return metrics

    @staticmethod
    def initialize_model(num_classes: int, weights='DEFAULT'):
        """
        Initialize DeepLabV3 model with ResNet50 backbone.
        
        Args:
            num_classes: Number of output classes (including background)
            weights: Pretrained weights specification
        
        Returns:
            Configured DeepLabV3 model
        """
        model = deeplabv3_resnet50(weights=weights)
        model.classifier = torchvision.models.segmentation.deeplabv3.DeepLabHead(2048, num_classes)
        return model

    def forward(self, x) -> Tensor:
        """Run forward pass through the model."""
        return self.model(x)['out']

    def predict_step(self, batch: dict, batch_idx, dataloader_idx=0):
        """Generate binary predictions from model output."""
        x = batch["image"]
        y_hat = self(x)
        return y_hat > 0

    def _get_metrics(self, phase: str) -> tuple:
        """
        Get appropriate metrics for the current phase.
        
        Args:
            phase: One of 'train', 'val', or 'test'
            
        Returns:
            Tuple of (segmentation_metrics, classification_metrics)
        """
        if phase == 'train':
            seg_metrics = self.train_seg_metrics
            cls_metrics = self.train_cls_metrics
        elif phase == 'val':
            seg_metrics = self.val_seg_metrics
            cls_metrics = self.val_cls_metrics
        elif phase == 'test':
            seg_metrics = self.test_seg_metrics
            cls_metrics = self.test_cls_metrics
        else:
            raise ValueError(f"Invalid phase: {phase}")
        return seg_metrics, cls_metrics

    def _run_step(self, batch: dict, batch_idx, phase: str) -> Tensor:
        """
        Run a single training/validation/test step.
        
        Args:
            batch: Input data containing 'image' and 'segmentations'
            batch_idx: Index of current batch
            phase: Current phase ('train', 'val', 'test', or None)
            
        Returns:
            Loss tensor
        """
        x = batch["image"]
        y = batch["segmentations"]
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        y_hat = y_hat > 0
        if phase is not None:
            y = y.to(torch.bool)
            seg_metrics, cls_metrics = self._get_metrics(phase)

            for metric in seg_metrics.values():
                metric.update(y_hat, y)
            for metric in cls_metrics.values():
                metric.update(y_hat, y)

            self.log(f"{phase}/loss", loss, prog_bar=True, on_epoch=True, on_step=False, batch_size=len(x))

        return loss

    def training_step(self,
                      batch: dict,
                      batch_idx):
        """Execute training step with loss calculation."""
        loss = self._run_step(batch, batch_idx, phase='train')
        return loss

    def log(self, name, value, *args, log_to_datamint=False, **kwargs):
        """
        Log metrics to both Lightning and optionally Datamint.
        
        Args:
            name: Metric name
            value: Metric value
            log_to_datamint: Whether to also log to Datamint experiment
        """
        super().log(name, value, *args, **kwargs)
        if self.experiment is not None and log_to_datamint:
            try:
                if isinstance(value, torch.Tensor):
                    value = value.item()
                self.experiment.log_metric(name, value)
            except Exception as e:
                print(f"ERROR: Failed to log metric {name}: {e}")

    def validation_step(self, batch: dict, batch_idx):
        """Execute validation step and update metrics."""
        self._run_step(batch, batch_idx, phase='val')

    def test_step(self, batch: dict, batch_idx):
        """Execute test step and update metrics."""
        self._run_step(batch, batch_idx, phase='test')

    def configure_optimizers(self):
        """Configure Adam optimizer with weight decay."""
        return torch.optim.AdamW(self.parameters(), lr=2e-4, weight_decay=1e-3)

    def _compute_metrics_end(self, phase: str):
        """
        Compute and log all metrics at the end of an epoch.
        
        Args:
            phase: Current phase ('train', 'val', or 'test')
        """
        seg_metrics, cls_metrics = self._get_metrics(phase)
        for name, metric in seg_metrics.items():
            self.log(f"{phase}/{name}", metric.compute(), on_epoch=True, on_step=False, log_to_datamint=True)
            metric.reset()
        for name, metric in cls_metrics.items():
            self.log(f"{phase}/{name}", metric.compute(), on_epoch=True, on_step=False, log_to_datamint=True)
            metric.reset()

    def on_train_epoch_end(self):
        """Compute and log training metrics at epoch end."""
        self._compute_metrics_end('train')

    def on_validation_epoch_end(self):
        """Compute and log validation metrics at epoch end."""
        self._compute_metrics_end('val')

    def on_test_epoch_end(self):
        """Compute and log test metrics at epoch end."""
        self._compute_metrics_end('test')


In [6]:
# Initialize our model with the number of classes detected from the dataset
model = MyModel(num_classes=len(Dtrain.segmentation_labels_set),
                experiment=exp)

## Training

We'll set up the training process with:
- Model checkpointing to save the best performing model
- GPU acceleration for faster training
- Validation after each epoch to monitor performance

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint

# Define checkpoint callback to save best models based on validation loss
checkpoint_callback = ModelCheckpoint(
    monitor='val/loss',          # Monitor validation loss
    filename='boneseg-{epoch:02d}-{val/loss:.4f}',
    save_top_k=1,                # Save only the best model
    mode='min',                  # Lower val loss is better
    save_last=False,              # Also save the last model
    verbose=False,
    auto_insert_metric_name=False,
)

save_last_epoch = ModelCheckpoint()

# Set up Lightning trainer with appropriate hardware acceleration
trainer = L.Trainer(
    accelerator='gpu',
    max_epochs=60,
    callbacks=[checkpoint_callback, save_last_epoch],
)

# Start model training
trainer.fit(model,
            train_dataloaders=train_dataloader,
            # Ideally, we should use be a separated validation set, but for simplicity we use the test set:
            val_dataloaders=test_dataloader
            )

In [ ]:
# Run final evaluation on test dataset to get performance metrics
trainer.test(model, dataloaders=test_dataloader) 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/Accuracy       │    0.7333333492279053     │
│       test/F1Score        │    0.6599999666213989     │
│      test/Precision       │     0.800000011920929     │
│        test/Recall        │    0.6000000238418579     │
│     test/Specificity      │    0.4000000059604645     │
│         test/iou          │    0.48983272910118103    │
│         test/loss         │    0.19067145884037018    │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.19067145884037018,
  'test/iou': 0.48983272910118103,
  'test/Recall': 0.6000000238418579,
  'test/Precision': 0.800000011920929,
  'test/F1Score': 0.6599999666213989,
  'test/Specificity': 0.4000000059604645,
  'test/Accuracy': 0.7333333492279053}]

In [ ]:
# Display summary of all logged metrics from the experiment
exp.summary_log['metrics']

{'test/Sensitivity': 0.6000000238418579,
 'test/Positive Predictive Value': 0.800000011920929,
 'test/F1Score': 0.6599999666213989,
 'test/Accuracy': 0.7333333492279053}

In [ ]:
# Finalize the experiment to complete logging
exp.finish()

After finishing, we can check the experiment in the Datamint web app. The dashboard provides visual tracking of metrics.

## Predict

Now that we have a trained model, let's generate predictions on the test dataset and visualize the results.

In [ ]:
# Generate predictions on the test dataset
predictions_batches: list[Tensor] = trainer.predict(model, test_dataloader)
# list of tensors. each tensor has shape (batch_size, #classes, H, W)

In [ ]:
# Visualize model predictions on test images

k = 0
for prediction_batch in predictions_batches:
    # prediction_batch is a tensor of shape (batch_size, #classes, H, W)
    for prediction in prediction_batch:
        item = Dtest[k]
        image = item['image']

        segs = item['segmentations']

        # Show predicted segmentation masks overlaid on the image
        prediction_mask = prediction[1:] == 1
        image_with_prediction = draw_masks(image,
                                           masks=prediction_mask)

        show(image_with_prediction)
        k += 1